In [ ]:
import pandas as pd
from urllib import request
import json
from sklearn.cluster import KMeans
import folium

df = pd.read_csv('course.csv')
print(df.head())
df = df[df['Borough'] != 'Not assigned']
df['index'] = range(df.shape[0])
df = df.set_index('index')
for i in range(df.shape[0]):
    if df.at[i,'Neighbourhood'] == 'Not assigned':
        df.at[i, 'Neighbourhood'] = df.at[i,'Borough']
for i in range(df.shape[0]):
    p = df.at[i,'Postcode']
    if p == '':
        continue
    else:
        for j in range(i + 1, df.shape[0]):
            q = df.at[j, 'Postcode']
            if p == q:
                df.at[i, 'Neighbourhood'] = df.at[i, 'Neighbourhood'] + ',' + df.at[j, 'Neighbourhood']
                df.at[j, 'Postcode'] = ''
df = df[df['Postcode'] != '']
df['index'] = range(df.shape[0])
df = df.set_index('index')
print(df.shape)

df['Latitude'] =''
df['Longitude'] = ''
dfll = pd.read_csv('Geospatial_Coordinates.csv')
for i in range(df.shape[0]):
    post = df.at[i,'Postcode']
    for j in range(dfll.shape[0]):
        if post == dfll.at[j,'Postal Code']:
            df.at[i, 'Latitude'] = dfll.at[j, 'Latitude']
            df.at[i, 'Longitude'] = dfll.at[j, 'Longitude']
            break

def get_total_result(Latitude,Longitude):
    id = 'UCLMNMBNOSE40D4FPNQMCDR3JHNV1ENATZOF23ZV0JRVTSVH'
    pw = 'C2HEYAZGPROYQH51F0Y344XSQBIIB2ZCJ0OJ1XAC5SK2KXMA'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v=20180201'.\
        format(id, pw, Latitude, Longitude)
    rq = request.Request(url)
    res = request.urlopen(rq)
    respoen = res.read()
    result = str(respoen, encoding="utf-8")
    js = json.loads(result)
    return js['response']['totalResults']
def get_total_distance(Latitude,Longitude):
    id = 'UCLMNMBNOSE40D4FPNQMCDR3JHNV1ENATZOF23ZV0JRVTSVH'
    pw = 'C2HEYAZGPROYQH51F0Y344XSQBIIB2ZCJ0OJ1XAC5SK2KXMA'
    url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v=20180201'.\
        format(id, pw, Latitude, Longitude)
    rq = request.Request(url)
    res = request.urlopen(rq)
    respoen = res.read()
    result = str(respoen, encoding="utf-8")

    try:
        js = json.loads(result)
        a = js['response']['venues'][0]['location']['distance']
    except IndexError:
        a = 0
    else:
        js = json.loads(result)
        a = js['response']['venues'][0]['location']['distance']
    return a

df['total_result'] = ''
df['distance'] = ''
df['total_result'] = df.apply(lambda row: get_total_result(row['Latitude'],row['Longitude']), axis=1)
df['distance'] = df.apply(lambda row: get_total_distance(row['Latitude'],row['Longitude']), axis=1)

X = df[['total_result','distance']]
k_means = KMeans(init = "k-means++", n_clusters = 3, n_init = 12)
k_means.fit(X)
k_means_labels = k_means.labels_
df['lable'] = ''
for i in range(len(k_means_labels)):
    df.at[i,'lable'] = k_means_labels[i]
venues_map = folium.Map(location=[43.70,-79.35], zoom_start=12)
df0 = df[df['lable'] == 0]
df0['index'] = range(df0.shape[0])
df0 = df0.set_index('index')
lat = df0['Latitude']
lng = df0['Longitude']
for i in range(len(lat)):
    a = lat[i]
    b = lng[i]
    folium.Marker(
        [a, b],
        popup='BEDOKVILLE',
    ).add_to(venues_map)
df1 = df[df['lable'] == 1]
df1['index'] = range(df1.shape[0])
df1 = df1.set_index('index')
lat = df1['Latitude']
lng = df1['Longitude']
for i in range(len(lat)):
    a = lat[i]
    b = lng[i]
    folium.Marker(
        [a, b],
        popup='Timberline Lodge',
        icon=folium.Icon(color='green')
    ).add_to(venues_map)
df2 = df[df['lable'] == 2]
df2['index'] = range(df2.shape[0])
df2 = df2.set_index('index')
lat = df2['Latitude']
lng = df2['Longitude']
for i in range(len(lat)):
    a = lat[i]
    b = lng[i]
    folium.Marker(
        [a, b],
        popup='Some Other Location',
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(venues_map)
file_path = r'C:\\Users\whl\Desktop\工业用地.html'
venues_map.save(file_path)     # 保存为html文件
import webbrowser
webbrowser.open(file_path)